In [2]:
import pandas as pd
import numpy as np
import re
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

In [3]:
tweets = pd.read_csv('tweets.csv')

In [4]:
tweets = tweets[~tweets.user_id.isna()]

tweets.loc[:,'isRT'] = tweets.text.apply(lambda x: False if pd.isna(x) else True if re.match(r'^RT @',x) else False)

In [5]:
tweets.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6296494 entries, 0 to 6296493
Data columns (total 19 columns):
Unnamed: 0             int64
id                     int64
text                   object
user_id                float64
in_reply_to_user_id    bool
retweeted_status_id    bool
retweet_count          float64
reply_count            float64
favorite_count         float64
num_hashtags           float64
num_urls               float64
num_mentions           float64
created_at             object
timestamp              object
crawled_at             object
updated                object
Cat                    object
bot                    bool
isRT                   bool
dtypes: bool(4), float64(7), int64(2), object(6)
memory usage: 792.6+ MB


In [6]:
len(tweets), 'tweets'

(6296494, 'tweets')

In [7]:
#tweets_humans = tweets[tweets.Cat=='Humans']
#display(tweets_humans.columns)

#first = True
def getTimes(times):
    if(len(times) == 0):
        return np.array([0,0,0,0])
    return np.array([times.mean().seconds, times.median().seconds, times.std().seconds, times.quantile(0.25).seconds, times.quantile(0.75).seconds])

def getDist(var):
    if(len(var) == 0):
        return np.array([0,0,0,0])
    return np.array([var.mean(), var.median(), var.std(), var.quantile(0.25), var.quantile(0.75)])

def getWeekAv(tw):
    return len(tw) / len(tw.weekNum.unique())

def getDataTweets(twU):
    twU = twU.sort_values(by='timestamp', ascending=0).fillna(0) #tweets del usuario
    dates = pd.to_datetime(twU.timestamp) #aca lo que hago es guardar la columnoa como datetime

    twU.loc[:,'dayOfWeek'] = dates.dt.dayofweek # dia de la semana
    twU.loc[:,'hour'] = dates.dt.hour #hora del tweet
    twU.loc[:,'weekNum'] = dates.apply(lambda x: str(x.week) + '-' + str(x.year))

    tweetsUser = twU[twU.isRT==False] #tweets propios del usuario
    tweetsRT = twU[twU.isRT==True] #retweets

    twU.loc[:,'interval'] = pd.to_datetime(twU.timestamp.shift(1)) - pd.to_datetime(twU.timestamp)
    times = pd.to_datetime(twU.timestamp.shift(1)) - pd.to_datetime(twU.timestamp) # calculo distancia entre tweets

    tweetsUser.loc[:,'interval'] = pd.to_datetime(tweetsUser.timestamp.shift(1)) - pd.to_datetime(tweetsUser.timestamp)
    timesUser = pd.to_datetime(tweetsUser.timestamp.shift(1)) - pd.to_datetime(tweetsUser.timestamp) # calculo distancia entre tweets

    tweetsRT.loc[:,'interval'] = pd.to_datetime(tweetsRT.timestamp.shift(1)) - pd.to_datetime(tweetsRT.timestamp)
    timesRT = pd.to_datetime(tweetsRT.timestamp.shift(1)) - pd.to_datetime(tweetsRT.timestamp) # calculo distancia entre tweets
    
    mediaSemanas = np.around(twU.groupby('dayOfWeek').apply(getWeekAv), 6)
    
    #ver word_to_vect
    result = np.concatenate([
        [
            twU.iloc[0].user_id,
            twU.iloc[0].bot,
            twU.iloc[0].Cat,
            twU.id.count(),
            len(twU.weekNum.unique())
        ],
        mediaSemanas,
        getTimes(times),
        getTimes(timesRT),
        getTimes(timesUser),
        getDist(tweetsUser.favorite_count) * 10000,
        getDist(tweetsUser.retweet_count) * 10000,
        getDist(tweetsUser.num_mentions) * 10000,
        getDist(tweetsUser.reply_count) * 10000,

    ])
    
    return result
#         display(twU.head())
    #time_median, time_std, max_cuart, min_cuart, media_lun, media_martes, dispercion_horario
    #media_palabras
    #cantidad de palabras unicas

    #cantidad_usr que retwittea / cantidad de retweets

# res = pd.concat([
#     tweets[tweets.bot==True].iloc[0:6000],
#     tweets[tweets.bot==False].iloc[0:6000]
# ]).groupby('user_id').apply(getDataTweets)

res = tweets.groupby('user_id').apply(getDataTweets)
cols = np.concatenate(
    [
        
        [x+'_mean',x+'_median', x+'_std', x+'_fq', x+'_tq'] for x in
        ['times','timesRT','timesUser','favorite_count','retweet_count','num_mentions','reply_count']
    ]
)
cols = np.concatenate([
    ['user_id', 'bot', 'Cat', 'tweetsCount', 'semanas', 'lun', 'mar', 'mie', 'jue', 'vie', 'sab', 'dom'],cols
])
res = pd.DataFrame(
    [x for x in res], columns = cols
)
res.index = res.user_id

print(len(res))

5995


In [8]:
display(res.head())
print(res.columns)

,user_id,bot,Cat,tweetsCount,semanas,lun,mar,mie,jue,vie,...,num_mentions_mean,num_mentions_median,num_mentions_std,num_mentions_fq,num_mentions_tq,reply_count_mean,reply_count_median,reply_count_std,reply_count_fq,reply_count_tq
user_id,,,,,,,,,,,,,,,,,,,,,
678033.0,678033.0,False,Humans,3195,27,20.56,24.32,19.56,18.2,17.72,...,9019.78021978022,10000.0,8720.070307113143,0.0,10000.0,0.0,0.0,0.0,0.0,0.0
722623.0,722623.0,False,Humans,3201,89,6.678571,5.069767,6.321429,5.602273,4.951807,...,5439.72706595906,0.0,6453.941616006178,0.0,10000.0,0.0,0.0,0.0,0.0,0.0
755116.0,755116.0,False,Humans,3209,30,11.423077,16.666667,18.464286,31.448276,23.586207,...,9337.34939759036,10000.0,10512.28738550282,0.0,20000.0,0.0,0.0,0.0,0.0,0.0
755746.0,755746.0,False,Humans,3234,91,5.046512,5.227273,6.72619,5.116279,5.845238,...,6102.09601081812,10000.0,6578.193523455916,0.0,10000.0,0.0,0.0,0.0,0.0,0.0
785080.0,785080.0,False,Humans,3235,23,24.0,19.285714,27.047619,31.631579,29.6,...,7184.534270650263,10000.0,8842.028987750564,0.0,10000.0,0.0,0.0,0.0,0.0,0.0


Index(['user_id', 'bot', 'Cat', 'tweetsCount', 'semanas', 'lun', 'mar', 'mie',
       'jue', 'vie', 'sab', 'dom', 'times_mean', 'times_median', 'times_std',
       'times_fq', 'times_tq', 'timesRT_mean', 'timesRT_median', 'timesRT_std',
       'timesRT_fq', 'timesRT_tq', 'timesUser_mean', 'timesUser_median',
       'timesUser_std', 'timesUser_fq', 'timesUser_tq', 'favorite_count_mean',
       'favorite_count_median', 'favorite_count_std', 'favorite_count_fq',
       'favorite_count_tq', 'retweet_count_mean', 'retweet_count_median',
       'retweet_count_std', 'retweet_count_fq', 'retweet_count_tq',
       'num_mentions_mean', 'num_mentions_median', 'num_mentions_std',
       'num_mentions_fq', 'num_mentions_tq', 'reply_count_mean',
       'reply_count_median', 'reply_count_std', 'reply_count_fq',
       'reply_count_tq'],
      dtype='object')


In [ ]:
# sns.pairplot(res[['times_mean','timesRT_mean','Cat']], hue='Cat', diag_kind="kde")


g = sns.scatterplot(x="times_mean", y="timesRT_mean", hue='Cat', data=res, alpha=0.5, size=2)

g.set(xlim=(0, 15000))

[(0, 15000)]

In [47]:

import plotly
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
plotly.__version__

init_notebook_mode(connected=True)

def plotearVars(variables):
    colors = {'Humans':'red', 'SocialSpambot1':'blue', 'SocialSpambot2':'green', 'SocialSpambot3':'black'}
    trace1 = go.Scatter3d(
        x=res[variables[0][0]],
        y=res[variables[1][0]],
        z=res[variables[2][0]],
        mode='markers',
        marker=dict(
            size=4,
            color=res['Cat'].apply(lambda x: colors[x]),
            line=dict(
                color='rgba(255,255,255,0.2)',
                width=0
            ),
            opacity=0.3
        )
    )
    layout = go.Layout(
        margin=dict(
            l=0,
            r=0,
            b=0,
            t=0
        ),
        scene = dict(
            xaxis = dict(
                nticks=4,
                range = (lambda: None if len(variables[0])==1 else variables[0][1])(),
                title=variables[0][0],
            ),
            yaxis = dict(
                range = (lambda: None if len(variables[1])==1 else variables[1][1])(),
                title=variables[1][0],
            ),
            zaxis = dict(
                range = (lambda: None if len(variables[2])==1 else variables[2][1])(),
                title= variables[2][0],
            ),
        )
    )
    print(  )
    fig = go.Figure(data=[trace1], layout=layout)
    iplot(fig)
    
plotearVars([
    ['times_median', [0,10000]],
    ['lun', [0,100]],
    ['timesUser_tq'],
])

In [59]:

#             title='times median',
#         ),
#         yaxis = dict(
#             range = [0,10000],
#             title='timesRT_mean',
#         ),
#         zaxis = dict(
#             range = [0,2000],
#             title='timesUser_tq',


plotearVars([
    ['times_tq'],
    ['favorite_count_mean', [0, 60000]],
    ['timesUser_tq'],
])

In [67]:
# res.to_csv('features_generadas.csv')